In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE
import time



In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df= pd.read_csv('/content/gdrive/MyDrive/data/master_Jaf_edited.csv')

In [ ]:
df['sex'] = df['sex'].replace({'male': 1, 'female': 0})
df.drop(df[(df['sex'] == 'other') | (df['sex'] == 'undefined')].index, axis=0, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# create a LabelEncoder object
le = LabelEncoder()

# encode the values of the 'specific_feature' column in the df
df['smoking'] = le.fit_transform(df['smoking'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# create a LabelEncoder object
le = LabelEncoder()

# encode the values of the 'specific_feature' column in the df
df['working'] = le.fit_transform(df['working'])

In [ ]:
df.drop(columns=['prescription_medication', 'public_transport_count'], inplace=True)
df.head()

,sex,age,bmi,smoking,alcohol,cannabis,amphetamines,cocaine,contacts_count,working,...,liver_disease,compromised_immune,heart_disease,lung_disease,diabetes,hiv_positive,hypertension,other_chronic,nursing_home,health_worker
0,1,20_30,27.7,8,NaN,NaN,NaN,NaN,NaN,5,...,0,0,0,0,0,0,0,0,0,0
1,0,30_40,21.6,8,NaN,NaN,NaN,NaN,NaN,5,...,0,0,0,0,0,0,0,0,0,0
2,1,90_100,27.7,8,NaN,NaN,NaN,NaN,NaN,5,...,0,1,0,1,1,0,1,0,0,0
3,1,60_70,32.4,8,NaN,NaN,NaN,NaN,NaN,5,...,0,0,0,0,1,0,1,0,0,0
4,1,30_40,25.4,8,NaN,NaN,NaN,NaN,NaN,5,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = df.dropna()

In [ ]:
# Assuming 'df1' is a DataFrame that should be created from 'data' after loading
df1 = df.copy()

# Data Preprocessing: Dropping specified columns
df1.drop(columns=['hiv_positive', 'smoking', 'working', 'alcohol', 'cocaine', 'amphetamines', 'heart_disease', 'contacts_count', 'bmi', 'cannabis', 'age'], inplace=True)


In [ ]:
# Splitting the data into input features and output variable
X = df1.drop(['covid19_positive'], axis=1)
y = df1['covid19_positive']


In [ ]:
# Resampling the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [ ]:
# Splitting the resampled data into train, validation, and test sets
train_x, temp_x, train_y, temp_y = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
validation_x, test_x, validation_y, test_y = train_test_split(temp_x, temp_y, test_size=0.5, random_state=42)


In [ ]:
# Class distribution reporting
print("Training set size:", train_x.shape[0], "| COVID-19 Positive:", np.sum(train_y), "| COVID-19 Negative:", len(train_y) - np.sum(train_y))
print("Validation set size:", validation_x.shape[0], "| COVID-19 Positive:", np.sum(validation_y), "| COVID-19 Negative:", len(validation_y) - np.sum(validation_y))
print("Testing set size:", test_x.shape[0], "| COVID-19 Positive:", np.sum(test_y), "| COVID-19 Negative:", len(test_y) - np.sum(test_y))

Training set size: 975727 | COVID-19 Positive: 487546 | COVID-19 Negative: 488181
Validation set size: 209084 | COVID-19 Positive: 104697 | COVID-19 Negative: 104387
Testing set size: 209085 | COVID-19 Positive: 104705 | COVID-19 Negative: 104380


In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.5 MB/s eta 0:00:00


In [ ]:
# Implementing GA for hyperparameter tuning on the validation set
from deap import base, creator, tools, algorithms
import random


In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Defining the AdaBoostClassifier setup and hyperparameter optimization with DEAP
def eval_adaboost(individual):
    n_estimators, learning_rate = individual
    clf = AdaBoostClassifier(n_estimators=int(n_estimators), learning_rate=learning_rate)
    return (np.mean(cross_val_score(clf, validation_x, validation_y, cv=3)),)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_n_estimators", random.randint, 50, 400)
toolbox.register("attr_learning_rate", random.uniform, 0.01, 2)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_n_estimators, toolbox.attr_learning_rate), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", eval_adaboost)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutPolynomialBounded, low=[50, 0.01], up=[400, 2], indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
# Running the GA
population = toolbox.population(n=30)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=True)

best_individual = tools.selBest(population, k=1)[0]
best_n_estimators, best_learning_rate = best_individual
print(f"Best Hyperparameters: n_estimators={best_n_estimators}, learning_rate={best_learning_rate}")


KeyboardInterrupt: 

In [ ]:
# Final Model Training and Evaluation
final_model = AdaBoostClassifier(n_estimators=int(best_n_estimators), learning_rate=best_learning_rate)
final_model.fit(train_x, train_y)


In [ ]:
# Predicting on the test set
y_pred = final_model.predict(test_x)
accuracy = accuracy_score(test_y, y_pred)
precision = precision_score(test_y, y_pred)
recall = recall_score(test_y, y_pred)
f1 = f1_score(test_y, y_pred)
auc_roc = roc_auc_score(test_y, final_model.predict_proba(test_x)[:, 1])

# Output the evaluation metrics
print(f"Final Test Accuracy: {accuracy}")
print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}, AUC-ROC: {auc_roc}")
